In [1]:
import nltk
import pandas as pd
from textblob import TextBlob
import logging
from gensim import corpora, models, similarities
from pprint import pprint
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3
import lda

In [2]:
ltr=pd.read_csv('/Users/shani16/fletcher/data/ltr2.csv')

/Users/shani16/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (5,7,10,11,13,14,15,16,17,22,24,25,26,27,29,32,33,34,35,36,39,40,41,44,45,57,58,61,68,77,78,79,80,82,107,108,144,145,146,147,154,155,156,157,158,160,161,162,163,164,166,168,171,173,174,175,176,177,179) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
ltr.loc[ltr['Translation to English for: LTR Comment'].isnull(),'Translation to English for: LTR Comment'] = ltr['LTR Comment']

In [4]:
ltr=ltr[(ltr['Survey Type'] == "Insights")]
det= ltr[(ltr['LTR'] <= 6)]
df=det[['email', 'Translation to English for: LTR Comment']]
df.columns=['email', 'comment']
df=pd.DataFrame(df.dropna())

In [5]:
df.head()

,email,comment
0,mban@bpba.com.ar,I would be interested they can solucuonar inco...
6,simone.barbosa@lojasrenner.com.br,Meet as soon as the so-called demands. Improve...
8,mdunn@wellington.com,It has been a very unsatisfactory experience w...
9,andre_doles@gap.com,Not leave your customers hanging when they nee...
10,cecilia.shea@sabic-ip.com,"Contract Management, strategic planning & regu..."


In [6]:
df1=df

In [7]:
df1=df1.replace({'\r': ' '}, regex=True)
df1=df1.replace({'\xff': ' '}, regex=True)
df1=df1.replace({'\xfa': ' '}, regex=True)
df1=df1.replace({'\x84': ' '}, regex=True)
df1=df1.replace({'\x94': ' '}, regex=True)
df1=df1.replace({'\x87': ' '}, regex=True)
df1=df1.replace({'\xc60': ' '}, regex=True)
df1=df1.replace({'\xa2lio': ' '}, regex=True)
df1=df1.replace({'\xc6o': ' '}, regex=True)
# df2=df1.comment.tolist()
# tok=[nltk.sent_tokenize(s) for s in df2]

In [8]:
df1.head()

,email,comment
0,mban@bpba.com.ar,I would be interested they can solucuonar inco...
6,simone.barbosa@lojasrenner.com.br,Meet as soon as the so-called demands. Improve...
8,mdunn@wellington.com,It has been a very unsatisfactory experience w...
9,andre_doles@gap.com,Not leave your customers hanging when they nee...
10,cecilia.shea@sabic-ip.com,"Contract Management, strategic planning & regu..."


In [9]:
df1['token']=df1['comment'].apply(word_tokenize)

In [10]:
df1.head()

,email,comment,token
0,mban@bpba.com.ar,I would be interested they can solucuonar inco...,"[I, would, be, interested, they, can, solucuon..."
6,simone.barbosa@lojasrenner.com.br,Meet as soon as the so-called demands. Improve...,"[Meet, as, soon, as, the, so-called, demands, ..."
8,mdunn@wellington.com,It has been a very unsatisfactory experience w...,"[It, has, been, a, very, unsatisfactory, exper..."
9,andre_doles@gap.com,Not leave your customers hanging when they nee...,"[Not, leave, your, customers, hanging, when, t..."
10,cecilia.shea@sabic-ip.com,"Contract Management, strategic planning & regu...","[Contract, Management, ,, strategic, planning,..."


In [11]:
df_new = df1.groupby('email').token.apply(lambda x: pd.DataFrame(x.values[0])).reset_index().drop('level_1', axis = 1)
df_new.columns = ['email','sentence']

In [12]:
stopwords = nltk.corpus.stopwords.words('english')
stopwords += ['.', ',', '(', ')', "'", '"', 'ca']

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer=CountVectorizer(stop_words=stopwords)
counts=count_vectorizer.fit_transform(df_new['sentence'])

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
count_vectorizer1=TfidfVectorizer(stop_words=stopwords)
counts1=count_vectorizer1.fit_transform(df_new['sentence'])

In [18]:
test=pd.DataFrame(df_new['sentence'])

In [25]:
test.to_csv('words.csv')

In [15]:
feature= counts.toarray()
feature_df = pd.DataFrame(feature, columns=count_vectorizer.get_feature_names())
feature_df[:1]

,00137422,06,10,100,1000,10000,1000s,11,12,13,...,yesterday,yet,youd,youtube,youve,zeitl,zero,zone,zsecure,zuk
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
word_frequency = feature_df.sum().reset_index()
word_frequency.columns=['word', 'freq']

word_frequency=word_frequency.sort(columns='freq', ascending= False)
word_frequency[:20]



/Users/shani16/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,word,freq
3112,products,546
4009,support,449
3106,product,418
476,better,258
1983,improve,243
2183,issues,151
4385,us,146
4390,use,144
1004,customer,144
4544,work,135


In [ ]:
feature1= counts1.toarray()
feature_df1 = pd.DataFrame(feature1, columns=count_vectorizer.get_feature_names())
feature_df1[:1]

In [ ]:
word_frequency1 = feature_df1.sum().reset_index()
word_frequency1.columns=['word', 'freq']

word_frequency1=word_frequency1.sort(columns='freq', ascending= False)
word_frequency1[:20]

## Clusters K Means

In [ ]:
from sklearn.cluster import KMeans
model = KMeans (n_clusters=30, init= 'k-means++', max_iter=100, n_init=1, verbose=3)

In [ ]:
model = KMeans (n_clusters=11, init= 'k-means++', max_iter=100, n_init=1 ,verbose=3)
model.fit(counts) #add sparse matrix 

In [ ]:
#sorting in descending order ie: -1
#giving top ten words ie: :10
print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = count_vectorizer.get_feature_names()
for i in range(10):
   print "Cluster %d:" % i,
   for ind in order_centroids[i, :10]:
       print ' %s' % terms[ind],
   print

In [ ]:
model1 = KMeans (n_clusters=11, init= 'k-means++', max_iter=100, n_init=1, verbose=3)
model1.fit(counts1) #add sparse matrix 

In [ ]:
clusters1=pd.Series(model1.labels_)

In [ ]:
#sorting in descending order ie: -1
#giving top ten words ie: :10
print("Top terms per cluster:")
order_centroids = model1.cluster_centers_.argsort()[:, ::-1]
terms = count_vectorizer.get_feature_names()
for i in range(9):
   print "Cluster %d:" % i,
   for ind in order_centroids[i, :10]:
       print ' %s' % terms[ind],
   print

In [ ]:
df_new['clusters1']=clusters1

In [ ]:
df_new.head()

In [ ]:
# cluster_1 = model.transform(counts)[:, 0]
# ind = np.argsort(cluster_1)[::-1][:50]
# df_new.iloc[ind]

In [ ]:
# cluster_1 = model.transform(counts)[:, 0]
# ind = np.argsort(cluster_1)[::-1][:50]
# df_new.iloc[ind]

In [ ]:
grouped = df_new.groupby('clusters1')

## LDA

In [ ]:
from __future__ import print_function
from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups

n_samples = 2000
n_features = 1000
n_topics = 10
n_top_words = 20


def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()


# Load the 20 newsgroups dataset and vectorize it. We use a few heuristics
# to filter out useless terms early on: the posts are stripped of headers,
# footers and quoted replies, and common English words, words occurring in
# only one document or in at least 95% of the documents are removed.


# data_samples = dataset.data
# print("done in %0.3fs." % (time() - t0))

# # Use tf-idf features for NMF.
print("Extracting tf-idf features for NMF...")
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, #max_features=n_features,
                                   stop_words='english')
t0 = time()
tfidf = tfidf_vectorizer.fit_transform(df_new['sentence'])
print("done in %0.3fs." % (time() - t0))

# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=n_features,
                                stop_words='english')
t0 = time()
tf = tf_vectorizer.fit_transform(df_new['sentence'])
print("done in %0.3fs." % (time() - t0))

# Fit the NMF model
print("Fitting the NMF model with tf-idf features,"
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_topics, random_state=1, alpha=.1, l1_ratio=.5).fit(tfidf)
exit()
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model:")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

print("Fitting LDA models with tf features, n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=5,
                                learning_method='online', learning_offset=50.,
                                random_state=0)
t0 = time()
lda.fit(tf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Extracting tf-idf features for NMF...
done in 0.460s.
Extracting tf features for LDA...
done in 0.435s.
Fitting the NMF model with tf-idf features,n_samples=2000 and n_features=1000...
done in 2.221s.

Topics in NMF model:
Topic #0:
ca software zuk facing facility facilitate extremely externally external extent factor extensive extensions express exposure explanation fact factors feature fail
Topic #1:
products solutions zuk extent facility facilitate extremely externally external extensions extensive fact express exposure explanation explain facing factors factor expertise
Topic #2:
support service knowledge facilitate extremely externally external extent extensive extensions facing express exposure explanation explain experts facility fact expert factor
Topic #3:
product code answering favorable fault experts explain explanation exposure express extensions extensive extent external externally extremely facilitate facility facing expertise
Topic #4:
improve enhance external facing fac